In [1]:
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV
import re
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('covid_fake.csv')

In [5]:
df.head()

,title,text,source,label
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,Fake
1,NaN,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,Fake
2,NaN,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,Fake
3,NaN,The Corona virus is a man made virus created i...,JoanneWrightForCongress,Fake
4,NaN,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,Fake


In [9]:
df.shape

(1164, 4)

In [11]:
df['label'].value_counts()

label
TRUE    584
Fake    345
fake    230
Name: count, dtype: int64

In [13]:
df.loc[5:15]

,title,text,source,label
5,CORONA UNMASKED: Chinese Intelligence Officer ...,NaN,NaN,NaN
6,NaN,Urgent: Health Bulletin to the Public. Ministr...,Ministry of Health,Fake
7,NaN,"Pls tell ur families, relatives and friendsMOH...",NWLLAB,Fake
8,NaN,SERIOUS EXCELLENT ADVICE by Japanese doctors t...,Japanese doctors treating COVID-19 cases,Fake
9,Basic protective measures against the new coro...,Stay aware of the latest information on the CO...,https://www.who.int/emergencies/diseases/novel...,TRUE
10,NaN,The new Coronavirus may not show signs of infe...,Taiwan Experts,Fake
11,NaN,A vaccine meant for cattle can be used to figh...,facebook,Fake
12,NaN,Using a hair dryer to breathe in hot air can c...,Youtube,Fake
13,NaN,Corona virus before it reaches the lungs it re...,twitter,Fake
14,Exposing yourself to the sun or to temperature...,"You can catch COVID-19, no matter how sunny or...",https://www.who.int/emergencies/diseases/novel...,TRUE


In [15]:
df.isna().sum()

title     82
text      10
source    20
label      5
dtype: int64

In [17]:
df.loc[df['label'] == 'Fake', ['label']] = 'FAKE'
df.loc[df['label'] == 'fake', ['label']] = 'FAKE'
df.loc[df['source'] == 'facebook', ['source']] = 'Facebook'
df.text.fillna(df.title, inplace=True)
df.loc[5]['label']= 'FAKE'
df.loc[15]['label'] = 'TRUE'
df.loc[43]['label'] = 'FAKE'
df.loc[131]['label'] = 'TRUE'
df.loc[242]['label'] = 'FAKE'
#df = df.sample(frac=1).reset_index(drop=True)
df.title.fillna('missing', inplace=True)
df.source.fillna('missing', inplace=True)
df['title_text'] = df['title'] + '' + df['text']

In [19]:
df.isna().sum()

title         0
text          0
source        0
label         0
title_text    0
dtype: int64

In [21]:
df['label'].value_counts()

label
TRUE    586
FAKE    578
Name: count, dtype: int64

In [23]:
df.head()

,title,text,source,label,title_text
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,FAKE,Due to the recent outbreak for the Coronavirus...
1,missing,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,FAKE,missingHydroxychloroquine has been shown to ha...
2,missing,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,FAKE,missingFact: Hydroxychloroquine has been shown...
3,missing,The Corona virus is a man made virus created i...,JoanneWrightForCongress,FAKE,missingThe Corona virus is a man made virus cr...
4,missing,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,FAKE,missingDoesn’t @BillGates finance research at ...


In [25]:
df.shape

(1164, 5)

In [31]:
df['title_text'][1]

'missingHydroxychloroquine has been shown to have a 100% effective rate treating COVID-19. Yet Democrat Gretchen Whitmer is threatening doctors who prescribe it. If Trump is for something, Democrats are against it. They are okay with people dying if it means opposing Trump.'

In [35]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[\n]', '', text)
    text = text.lower()
    return text
df['title_text'] = df['title_text'].apply(preprocessor)
df['title_text'][1]

'missinghydroxychloroquine has been shown to have a 100 effective rate treating covid19 yet democrat gretchen whitmer is threatening doctors who prescribe it if trump is for something democrats are against it they are okay with people dying if it means opposing trump'

In [47]:
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None,
                        tokenizer=tokenizer_porter,
                        use_idf=True,
                        norm='l2',
                        smooth_idf=True)

X = tfidf.fit_transform(df['title_text'])
y = df['label'].values

In [50]:
X.shape

(1164, 27699)

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, \
test_size=0.3, shuffle=False)

clf = LogisticRegressionCV(cv=5, scoring='accuracy', random_state=0, n_jobs =- 1, \
verbose=0, max_iter=300)
clf.fit(X_train, y_train)

fake_news_model = open('fake_news_model.sav', 'wb')
pickle.dump(clf, fake_news_model)
fake_news_model.close()

In [57]:
filename = 'fake_news_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(X_test, y_test)

0.94

In [59]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = clf.predict(X_test)
print(" --- Test Set Results --- ")
print(classification_report(y_test, y_pred))

 --- Test Set Results --- 
              precision    recall  f1-score   support

        FAKE       0.92      0.92      0.92       132
        TRUE       0.95      0.95      0.95       218

    accuracy                           0.94       350
   macro avg       0.94      0.94      0.94       350
weighted avg       0.94      0.94      0.94       350

